In [1]:
 %matplotlib inline

In [1]:
import matplotlib.pyplot as plt
import cv2
from pathlib import Path
import os
from PIL import *
import matplotlib.image as mpimg
import numpy as np
from keras.preprocessing import image
import json
import random
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.layers import Dense, Dropout, Flatten
from keras.applications.resnet50 import ResNet50
from keras.applications.resnet import ResNet152
from keras.applications.inception_v3 import InceptionV3
import keras
from keras.models import Sequential,Input,Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D,GlobalAveragePooling2D, ReLU, MaxPool2D,InputLayer
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU   
from keras import optimizers, regularizers
from sklearn.metrics import classification_report
from keras.callbacks import TensorBoard
import datetime
import imgaug.augmenters as iaa
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.resnet50 import preprocess_input

import sys

Using TensorFlow backend.


In [5]:

def split_data(data_dict):
    trainset = []
    valset = []
    testset=[]
    for label, images in data_dict.items():
        random.shuffle(images)
        img_train, img_test = train_test_split(images, test_size=0.2)
        img_train, img_val = train_test_split(img_train,test_size=0.2)
        trainset = trainset + img_train
        valset = valset + img_val
        testset = testset + img_test
    
    return trainset, valset, testset

def create_data(images_dict):
    data = {
        0:[],
        1:[],
        2:[],
        3:[],
        4:[]
    }
    for label, img_paths in images_dict.items():
        for img_path in img_paths:
            img = image.load_img(img_path, target_size=(224,224))
            img = image.img_to_array(img)
#             print(img)
#             print('============================\n')
#             print(img.astype('uint8'))
# #             pyplot.imshow(img.astype('uint8'))
            data[label].append([img, label])

    return data


In [6]:
path_base = 'E:\\aptos\\labelsbase15.json'
# path_base = 'E:\\aptos\\labelsnovel15.json'
# path_base = 'E:\\aptos\\labelsval15.json'
classes = 5  #TODO:
with open(path_base, 'r') as f:
    data = json.load(f)

labels = np.array(data['image_labels'])
images = np.array(data['image_names'])
# print(images[0])
check = 'E:\\aptos\\resized_train_15\\163_right.jpg'
image_path = images[images ==check]
print(image_path)
labels[images == check]


image_path = 'E:\\aptos\\resized_train_15\\10_left.jpg'
# image_path = 'E:\\aptos\\resized_train_15\\16_left.jpg'

['E:\\aptos\\resized_train_15\\163_right.jpg']


In [13]:
img = image.load_img(image_path, target_size=(224,224))
img = image.img_to_array(img)
#             print(img)
#             print('============================\n')
#             print(img.astype('uint8'))
# #             pyplot.imshow(img.astype('uint8'))



  
print(img.max(), img.mean(),img.std(), img.min())

255.0 93.82242 61.414486 0.0


In [14]:
from keras.applications.resnet50 import preprocess_input

check = preprocess_input(img, mode='torch', data_format='channels_last')
print(check.max(), check.mean(), check.std(), check.min())

2.4110641 -0.35927162 1.0526711 -2.117904


In [81]:
base_model =ResNet50(weights='imagenet', include_top=True, input_shape=(224,224,3))
x = base_model.output
# x = GlobalAveragePooling2D()(x)
# x = Dropout(0.5)(x)
predictions = Dense(5, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

In [87]:
x = np.expand_dims(img, 0)

In [88]:
model.predict(x)

array([[0.19973259, 0.20001434, 0.20022984, 0.1996388 , 0.20038445]],
      dtype=float32)

In [109]:
import pandas as pd
df = pd.read_csv('E:\\aptos\\labels\\trainLabels15.csv')

In [112]:
for index, row in df.iterrows():
  print(row)
  break

image    10_left
level          0
Name: 0, dtype: object


In [4]:
def get_model(input_shape):
  
  base_model =ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
  #for layer in  base_model.layers[:10]:
    #layer.trainable = False
    #layer.padding='same'
 
  #for layer in  base_model.layers[10:]:
    #layer.trainable = True
    #layer.padding='same'
    
#   x = base_model.get_layer('avg_pool').output
  x = base_model.output
#   x = GlobalAveragePooling2D()(x)
  # x = BatchNormalization()(x)
#   x = Dropout(0.5)(x)

  x = Flatten() (x)
#   x = Dropout(0.5)(x)
  # x = Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.001))(x)
#   # x = BatchNormalization()(x)
#   x = Dropout(0.5)(x)
#   x = Dense(32, activation='relu')(x)
  # x = Dense(128, activation='relu')(x)
  # x = Dropout(0.5)(x)
#   x = Dense(2048, activation='relu')(x)
  x = Dense(512, activation='relu')(x)
  # x = LeakyReLU(alpha=0.1)(x)
    
  x = Dropout(0.5)(x)
  #x = Dense(5, activation='softmax')(x)
  #model = Model(base_model.input, x)
  predictions = Dense(5, activation='softmax')(x)
  model = Model(inputs=base_model.input, outputs=predictions)
  for layer in model.layers[:-2]:
    layer.trainable = False

  return model

In [5]:
model50 = get_model(input_shape=(224,224,3))
model50.summary()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.



C:\Users\clc87\Anaconda3\lib\site-packages\keras_applications\resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
____________________________________________________________________________________________

In [7]:
for layer in model50.layers[:165]:
      layer.trainable = False
for layer in model50.layers[165:]:
  layer.trainable = True
for i, layer in enumerate(model50.layers):
   print(i, layer.name, layer.trainable)

0 input_1 False
1 conv1_pad False
2 conv1 False
3 bn_conv1 False
4 activation_1 False
5 pool1_pad False
6 max_pooling2d_1 False
7 res2a_branch2a False
8 bn2a_branch2a False
9 activation_2 False
10 res2a_branch2b False
11 bn2a_branch2b False
12 activation_3 False
13 res2a_branch2c False
14 res2a_branch1 False
15 bn2a_branch2c False
16 bn2a_branch1 False
17 add_1 False
18 activation_4 False
19 res2b_branch2a False
20 bn2b_branch2a False
21 activation_5 False
22 res2b_branch2b False
23 bn2b_branch2b False
24 activation_6 False
25 res2b_branch2c False
26 bn2b_branch2c False
27 add_2 False
28 activation_7 False
29 res2c_branch2a False
30 bn2c_branch2a False
31 activation_8 False
32 res2c_branch2b False
33 bn2c_branch2b False
34 activation_9 False
35 res2c_branch2c False
36 bn2c_branch2c False
37 add_3 False
38 activation_10 False
39 res3a_branch2a False
40 bn3a_branch2a False
41 activation_11 False
42 res3a_branch2b False
43 bn3a_branch2b False
44 activation_12 False
45 res3a_branch2c False

In [158]:
  def get_model(input_shape):
    kernel_size = 3
    model = Sequential([
      InputLayer(input_shape=input_shape),
      Conv2D(32,kernel_size ),
      BatchNormalization(),
      ReLU(),
      MaxPooling2D(pool_size=(3,3), strides=(2,2)),
      Conv2D(64,kernel_size , input_shape=input_shape),
      BatchNormalization(),
      ReLU(),
      MaxPooling2D(pool_size=(3,3), strides=(2,2)),
      Conv2D(512,kernel_size , input_shape=input_shape),
      BatchNormalization(),
      ReLU(),
      GlobalAveragePooling2D(),
      Dense(5, activation='softmax'),
    ])
    return model

In [159]:
model = get_model((64,64,3))

In [160]:
model.summary()

Model: "sequential_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_74 (Conv2D)           (None, 62, 62, 32)        896       
_________________________________________________________________
batch_normalization_74 (Batc (None, 62, 62, 32)        128       
_________________________________________________________________
re_lu_74 (ReLU)              (None, 62, 62, 32)        0         
_________________________________________________________________
max_pooling2d_66 (MaxPooling (None, 30, 30, 32)        0         
_________________________________________________________________
conv2d_75 (Conv2D)           (None, 28, 28, 64)        18496     
_________________________________________________________________
batch_normalization_75 (Batc (None, 28, 28, 64)        256       
_________________________________________________________________
re_lu_75 (ReLU)              (None, 28, 28, 64)      